## Imports 

In [1]:
import common.feature_string as feature_string
import common.feature_num as feature_num
import common.feature_categorica as feature_categorica
import common.common_machine_learning as common
import common.my_pipeline as my_pipe
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import time

### Ignore Warnings 

In [2]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

## Constantes

In [3]:
TARGET = 'precio'
K = 3
RANDOM_SEMILLA = 3 # numero favorito (de la suerte)
XGB_RE_OBJECTIVE = 'reg:squarederror'
N_ITER = 15

In [11]:
t0 = time.time()
train = common.cargar_set_optimizado('sets_de_datos/train.csv', index_col = 0)
t1 = time.time()
print('Tiempo = {0:.2f} minutos'.format((t1-t0)/60))

Tiempo = 0.05 minutos


## Features agregables por registro (no necesita entrenamiento previo)

In [12]:
t0 = time.time()
feature_num.agregar_feature_fecha_numerica(train)
t1 = time.time()
print('Tiempo = {0:.2f} minutos'.format((t1-t0)/60))

Tiempo = 0.00 minutos


In [14]:
t0 = time.time()
train = feature_categorica.agregar_feature_one_hot_encoding(train)
t1 = time.time()
print('Tiempo = {0:.2f} minutos'.format((t1-t0)/60))

Tiempo = 0.09 minutos


Esto puede tardar unos minutos.  
Dima'pc = 3 minutos (aprox)

In [5]:
t0 = time.time()
feature_string.agregar_feature_todos_ref(train)
t1 = time.time()
print('Tiempo : {0:.2f} minutos'.format((t1-t0)/60))

Tiempo : 2.58 minutos


## Muestra y Target

In [4]:
X = train.drop([TARGET], axis = 1).copy()
y = train[TARGET].copy()

## Pipeline 

In [26]:
#pre_procesar_pipe = my_pipe.get_train_simple_pre_pipeline()
busqueda_pipe = Pipeline(steps = [
#    ('pre-procesar', pre_procesar_pipe),
    ('xgb_regressor', XGBRegressor(objective = XGB_RE_OBJECTIVE))
])  

In [ ]:
hyper_params = {
    "xgb_regressor__n_estimators": [650 ,850], # default 100,
    "xgb_regressor__learning_rate": [0.01, 0.05], # default 0.1
    "xgb_regressor__gamma": [0.2, 0.8],
    "xgb_regressor__lambda": [0.8, 1.2],
    "xgb_regressor__max_depth": [4, 7], # default 3 # < 15 columnas = cantidad columnas feature originales
    "xgb_regressor__subsample": [0.65, 0.45],
    "xgb_regressor__colsample_bytree": [0.65, 0.45],
    "xgb_regressor__min_child_weight": [3, 6]
}

busqueda = RandomizedSearchCV(
            busqueda_pipe, 
            param_distributions=hyper_params,
            random_state = RANDOM_SEMILLA,
            n_iter = N_ITER,
            cv=K, 
            verbose=1, 
            n_jobs=1, 
            return_train_score=True,
            iid = True
        )